# Workshop 07

<b>Obiettivo</b>: Realizzazione del modello 3D di <i><b>Villa Gazzotti Grimani</b></i>.

![VillaGazzotti_2007_07_18_1.jpg](files/Immagini/VillaGazzotti_2007_07_18_1.jpg)

In [1]:
# Importo le librerie:
import sys
sys.path.insert(0, 'C:\Users\Camilla Bianca\Documents\GGPL\lar-cc\larlib')
from larlib import *
import math
import csv

Evaluating fenvs.py..
...fenvs.py imported in 0.00698348469687 seconds


<hr>

## Struttura piano terra:

### - Stanza:

In [2]:
# Le misure sono in metri.
#
# X = larghezza; Y = altezza; Z = profondità
#
# Definiamo la funzione per costruire una stanza:
def stanza (x,y,z):
    cubo = CUBOID([x+1,y,z+1])
    cuboDaSottrarre = T([1,3])([0.5,0.5])(CUBOID([x,y,z]))
    return DIFFERENCE([cubo,cuboDaSottrarre])

# Esempio di stanza cubica:
VIEW(stanza(20,20,20))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DE4D420> >

![ws7_stanza.png](files/Immagini/ws7_stanza.png)

### - Pianta:

In [3]:
# Definiamo una funzione per creare la pianta del primo piano:
# (h = altezza muri)
def pianoTerra(h):
    # Creiamo le stanze della pianta: 
    # - Stanza rettangolare in alto, a destra e sinistra:
    stanzaPiccola = stanza(4,h,9)
    # - Stanza quadrata:
    stanzaQuadrata = stanza(9,h,9)
    # - Stanza quadrata in basso, a destra e sinsitra:
    stanzaGrande = stanza(12,h,12)
    # - Stanza ccentralo a forma di croce:
    stanzaCroce1 = DIFFERENCE([STRUCT([T(3)(4)(stanza(17,h,9)),T(1)(4)(stanza(9,h,17))]),T([1,3])([4.5,1])(CUBOID([9,h,16]))])
    stanzaCroce = DIFFERENCE([stanzaCroce1,T([1,3])([1,4.5])(CUBOID([16,h,9]))])
    # - Stanza dove saranno poste le scale interne:
    stanzetta = stanza(5,h,3.5)
    # Struttura del piano terra:
    pianoTerra1 = STRUCT([
        stanzaPiccola,
        T(1)(4.5)(stanzaQuadrata),
        T([1,3])([14,-4])(stanzaCroce),
        T(1)(31.5)(stanzaQuadrata),
        T(1)(41)(stanzaPiccola)
    ])
    pianoTerra = STRUCT([
        pianoTerra1,
        T(3)(9.5)(stanzaGrande),
        T([1,3])([12.5,9.5])(stanzetta),
        T([1,3])([27.5,9.5])(stanzetta),
        T([1,3])([33,9.5])(stanzaGrande),
    ])
    return pianoTerra

h = 7
VIEW(pianoTerra(h))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DE6CA80> >

![ws7_pianta.png](files/Immagini/ws7_pianta.png)

<hr>

### - Porte:

In [4]:
# Creiamo la struttura per le porte:
def porta():
    maniglia = JOIN(SPHERE(0.1)([18,36]))
    # Creiamo un blocco della dimensione della porta:
    p = CUBOID([1.2,3,0.5])
    buco = CUBOID([0.45,0.85,0.05])
    buchi = STRUCT([T([1,2])([0.1,0.1])(buco),T([1,2])([0.65,0.1]),(buco)])
    # Creiamo i dettagli della porta:
    pDaSottrarre = STRUCT([buchi,T(2)(1)(buchi),T(2)(1.95)(buchi),
                          T(3)(0.45)(buchi),T([2,3])([1,0.45])(buchi),T([2,3])([1.95,0.45])(buchi)])
    # Otteniamo la porta sottraendo al blocco iniziale p la struttura pDaSottrarre
    diff = DIFFERENCE([p,pDaSottrarre])
    return STRUCT([TEXTURE('Texture/legno4.jpg')(diff),
                   T([1,2,3])([0.05,1.55,-0.1])(TEXTURE('Texture/ottone1.jpg')(maniglia)),
                   T([1,2,3])([0.05,1.55,0.6])(TEXTURE('Texture/ottone1.jpg')(maniglia))])
    
VIEW(porta())

Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DE4DE40> >

![ws7_porta.png](files/Immagini/ws7_porta.png)

### - Fori per le porte:

In [6]:
# Eliminiamo dalla struttura i blocchi della grandezza delle porte:
#
# Blocco sull'asse X:
bloccoX = CUBOID([1.2,3,0.5])
# Blocchi sull'asse Z:
bloccoZ1 = CUBOID([37,3,1.2])
bloccoZ2 = CUBOID([12,3,1.2])
bloccoZ3 = CUBOID([22,3,1.2])
# Uniamo in un'unica struttura i buchi opportunamente traslati:
buchiPortePT = STRUCT([
    T([1,3])([4.5,4.4])(bloccoZ1),
    T([1,3])([17.5,11])(bloccoZ2),
    T([1,3])([12,17])(bloccoZ3),
    T([1,3])([8.5,9.5])(bloccoX),
    T([1,3])([36,9.5])(bloccoX),
    # porta ingresso:
    T([1,3])([22.4,13.5])(bloccoX),
    # porta retro:
    T([1,3])([22,-4])(CUBOID([2,4,0.5]))
])
# Otteniamo la struttura con le fessure per le porte:
pianoTerraBuchiPorte = DIFFERENCE([pianoTerra(h),buchiPortePT])

VIEW(pianoTerraBuchiPorte)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEBCC30> >

![ws7_foriPortePT.png](files/Immagini/ws7_foriPortePT.png)

### - Inserimento porte:

In [7]:
# Posizioniamo le porte nelle fessure:
#
# Ruotiamo la porta per posizionarla sui muri lungo l'asse z:
portaR = R([1,3])(PI/2)(porta())
# Creiamo una struttura che contenga tutte le porte nelle posizioni esatte:
portePT = STRUCT([
    T([1,3])([8.5,9.5])(porta()),
    T([1,3])([36,9.5])(porta()),
    T([1,3])([5,4.4])(portaR),
    T([1,3])([14.5,4.4])(portaR),
    T([1,3])([32,4.4])(portaR),
    T([1,3])([41.5,4.4])(portaR),
    T([1,3])([18.5,11])(portaR),
    T([1,3])([28,11])(portaR),
    T([1,3])([13,17])(portaR),
    T([1,3])([33.5,17])(portaR)
])
pianoTerraPorte = STRUCT([pianoTerraBuchiPorte,portePT])

VIEW(pianoTerraPorte)

Child 3
Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEBCB40> >

![ws7_insPortePT.png](files/Immagini/ws7_insPortePT.png)

<hr>

### - Finestre:

In [8]:
# Costruiamo una funzione per simulare il vetro:
def hex_material(color, light, trasparence):
    def hex_to_rgb(value):
        value = value.lstrip('#')
        lv = len(value)
        return map(lambda x: x/255., list(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3)))
    params = hex_to_rgb(color) + [.1] + \
                hex_to_rgb(light) + [trasparence] +\
                hex_to_rgb(light) + [.1] +\
                hex_to_rgb("#000000") + [.1] +\
                [100]
    return MATERIAL(params)
        
# Ci sono tre tipi di finestre, una più alta (grande), una quadrata (piccola),
# e quella ancora più grande per la facciata:
#
# Definiamo la funzione per la finestra alta (grande):
def finestraGrande():
    # Creiamo la lastra di vetro:
    f = CUBOID([1,2,0.1])
    finestra = hex_material("#ffffff", "#ffffff",0.3)(f)
    # Creiamo la cornice esterna:
    cornice = STRUCT([
        T([1,2,3])([-0.04,-0.04,0.1])(CUBOID([1.08,0.04,0.01])),
        T([1,3])([-0.04,0.1])(CUBOID([0.04,2,0.01])),
        T([1,3])([1,0.1])(CUBOID([0.04,2,0.01])),
        CUBOID([1.01,0.01,0.1]),
        CUBOID([0.01,2,0.1]),
        T([1,2])([1,0.01])(CUBOID([0.01,2,0.1])),
        T(1)(-0.14)(CUBOID([0.1,2,0.01])),
        T(1)(1.04)(CUBOID([0.1,2,0.01]))
    ])
    # Con MKPOLS creiamo la struttura triangolare superiore:
    verts1 = [[0,0,0],[0.1,0,0],[0.65,0.2,0],[1.2,0,0],[1.3,0,0],[0.65,0.3,0]]
    cells = [[1,2,3,6],[3,4,5,6]]
    pols = [[1],[2]]
    dettaglio1= OFFSET([0,0,0.1])(MKPOL([verts1,cells,pols]))
    verts2 = [[0.1,0,0],[0.15,0,0],[0.65,0.2,0],[1.2,0,0],[1.25,0,0],[0.65,0.25,0]]
    dettaglio2 = OFFSET([0,0,0.12])(MKPOL([verts2,cells,pols]))
    dettaglio3 = STRUCT([
        TEXTURE('Texture/pietra3.jpg')(T(1)(0.1)(CUBOID([1.1,0.04,0.1]))),
        TEXTURE('Texture/pietra2.jpg')(T(1)(0.11)(CUBOID([1.1,0.005,0.12]))),
        TEXTURE('Texture/pietra2.jpg')(T([1,2])([0.2,0.04])(CUBOID([0.9,0.005,0.12])))
    ])
    triangolo = STRUCT([
        TEXTURE('Texture/pietra2.jpg')(dettaglio1),
        TEXTURE('Texture/pietra1.jpg')(dettaglio2),
        dettaglio3
    ])
    # Creiamo i riccioli laterali:
    dettaglio4 = DIFFERENCE([
        T([2,3])([0.06,0.2])(R([1,3])(-PI/2)(CYLINDER([0.06,0.04])(20))),
        T(2)(0.1)(CUBOID([0.1,0.1,0.27]))
    ])
    dettaglio5 = T([2,3])([0.07,0.03])(R([1,3])(-PI/2)(CYLINDER([0.03,0.04])(20)))
    dettaglio6 = OFFSET([0.04,0,0])(MKPOL([
        [[0,0.1,0.02],[0,0.005,0.2],[0,0.04,0.2]],
        [[1,2,3]],
        [[1]]
    ]))
    ricciolo1 = T([1,2,3])([-0.14,1.75,0.1])(R([2,3])(-PI/2)(STRUCT([
        dettaglio4,
        dettaglio5,
        dettaglio6
    ])))
    ricciolo = STRUCT([ricciolo1,T(1)(0.05)(ricciolo1)])
    # Uniamo il tutto:
    return STRUCT([
        finestra,
        TEXTURE('Texture/pietra2.jpg')(cornice),
        T([1,2])([-0.15,2])(triangolo),
        TEXTURE('Texture/pietra3.jpg')(ricciolo),
        TEXTURE('Texture/pietra3.jpg')(T(1)(1.19)(ricciolo))
    ])

VIEW(finestraGrande())

# Prima di definire la funzione per la finestra piccola,
# definiamo due funzioni di appoggio usate nell'HW4 per creare un arco:
def ArchSurface(rr,w,h):
    x1=lambda k: rr*cos(k[0])
    y1=lambda k: rr*sin(k[0])
    x2=lambda k: (rr-w)*cos(k[0])
    y2=lambda k: (rr-w)*sin(k[0])
    z=lambda k: h
    
    Circle0=BEZIER(S1)([CONS([x1,y1,z])])
    Circle1=BEZIER(S1)([CONS([x2,y2,z])])
    return BEZIER(S2)([Circle0,Circle1])

def Arch(length,w,depth,angle):
    radius=(length/2)/cos(angle/2)
    domain2D=PROD([T(1)(angle/2)(INTERVALS(PI-angle)(16)),QUOTE([1])])
    domain3D=PROD([domain2D,QUOTE([1])])
    ArchSurf2D_0=ArchSurface(radius,w,0)
    ArchSurf2D_1=ArchSurface(radius,w,depth)
    SolidMap=BEZIER(S3)([ArchSurf2D_0,ArchSurf2D_1])
    #ceiling=MIN(2)(MAP(ArchSurf2D)(domain2D))
    return MAP(SolidMap)(domain3D)

# Definiamo ora la funzione per la finestra piccola:
def finestraPiccola():
    f = CUBOID([1,1,0.1])
    finestra = hex_material("#ffffff", "#ffffff",0.3)(f)
    cornice = STRUCT([
        T([1,3])([-0.04,0.1])(CUBOID([0.04,1,0.01])),
        T([1,3])([1,0.1])(CUBOID([0.04,1,0.01])),
        T([1,2,3])([-0.04,-0.04,0.1])(CUBOID([1.08,0.04,0.01])),
        T([1,2,3])([-0.04,1,0.1])(CUBOID([1.08,0.04,0.01])),
        CUBOID([0.01,1,0.1]),
        T(1)(1)(CUBOID([0.01,1,0.1])),
        CUBOID([1,0.01,0.1]),
        T(2)(1)(CUBOID([1,0.01,0.1]))
    ])
    cornice1 = STRUCT([
        T(1)([-0.2])(CUBOID([0.2,1,0.01])),
        T(1)(1)(CUBOID([0.2,1,0.01])),
        T([1,2])([-0.2,-0.1])(CUBOID([1.4,0.1,0.01])),
        T([1,2])([-0.2,1])(CUBOID([1.4,0.1,0.01]))
    ])
    return STRUCT([
        finestra,
        TEXTURE('Texture/pietraB.jpg')(cornice),
        TEXTURE('Texture/pietra2.jpg')(cornice1),
        TEXTURE('Texture/mattoni2.jpg')(T(1)(0.5)(Arch(1.5,0.3,0.02,0.7*PI)))
    ])
    
VIEW(finestraPiccola())

# Definiamo infine la funzione per la finestra grande per la facciata:
def finestraFacciata():
    # Creiamo la lastra di vetro:
    f = CUBOID([2,4,0.2])
    finestra = hex_material("#ffffff", "#ffffff",0.5)(f)
    # Creiamo la cornice:
    # - cornice esterna:
    corniceEst = STRUCT([
        T([1,2])([-0.2,-0.2])(CUBOID([2.4,0.2,0.01])),
        T(1)(-0.2)(CUBOID([0.2,4.2,0.01])),
        T(1)(2)(CUBOID([0.2,4.2,0.01])),
        T([1,2])([-0.2,4])(CUBOID([2.4,0.2,0.01]))
    ])
    # - cornice interna:
    corniceInt = STRUCT([
        # - parte più interna:
        T([1,2,3])([-0.05,-0.05,0.01])(CUBOID([2.1,0.05,0.01])),
        T([1,3])([-0.05,0.01])(CUBOID([0.05,4.05,0.01])),
        T([1,3])([2,0.01])(CUBOID([0.05,4.05,0.01])),
        T([1,2,3])([-0.05,4,0.01])(CUBOID([2.1,0.05,0.01])),
        # - parte centrale:
        T([1,2,3])([-0.1,-0.1,0.01])(CUBOID([2.2,0.025,0.001])),
        T([1,2,3])([-0.1,-0.1,0.01])(CUBOID([0.025,4.2,0.01])),
        T([1,2,3])([2.075,-0.1,0.01])(CUBOID([0.025,4.2,0.01])),
        T([1,2,3])([-0.1,4.075,0.01])(CUBOID([2.2,0.025,0.01])),
        # - parte più esterna:
        T([1,2,3])([-0.25,-0.25,0.01])(CUBOID([2.5,0.05,0.001])),
        T([1,2,3])([-0.25,-0.25,0.01])(CUBOID([0.05,4.5,0.01])),
        T([1,2,3])([2.2,-0.25,0.01])(CUBOID([0.05,4.5,0.01]))
    ])
    # Con MKPOL creiamo la struttura triangolare superiore:
    verts1 = [[0,0,0],[0.2,0,0],[1.2,0.5,0],[2.2,0,0],[2.4,0,0],[1.2,0.7,0]]
    cells = [[1,2,3,6],[3,4,5,6]]
    pols = [[1],[2]]
    dettaglio1= OFFSET([0,0,0.2])(MKPOL([verts1,cells,pols]))
    verts2 = [[0.2,0,0],[0.3,0,0],[1.2,0.3,0],[2.1,0,0],[2.2,0,0],[1.2,0.5,0]]
    dettaglio2 = OFFSET([0,0,0.1])(MKPOL([verts2,cells,pols]))
    dettaglio3 = TEXTURE('Texture/pietra2.jpg')(CUBOID([2.4,0.05,0.2]))
    triangolo = T(2)(0.2)(STRUCT([
        TEXTURE('Texture/pietra2.jpg')(dettaglio1),
        TEXTURE('Texture/pietra1.jpg')(dettaglio2),
        dettaglio3
    ]))
    # Creiamo i riccioli laterali:
    dettaglio4 = DIFFERENCE([
        T([2,3])([0.06,0.2])(R([1,3])(-PI/2)(CYLINDER([0.1,0.08])(20))),
        T(2)(0.1)(CUBOID([0.2,0.3,0.5]))
    ])
    dettaglio5 = T([1,2,3])([0.015,0.015,-0.2])(R([1,3])(-PI/2)(CYLINDER([0.06,0.05])(20)))
    dettaglio6 = T([1,2,3])([0.015,-0.05,-0.3])(OFFSET([0.05,0,0])(MKPOL([
        [[0,0.1,0.05],[0,0.05,0.5],[0,0.1,0.5]],
        [[1,2,3]],
        [[1]]
    ])))
    ricciolo1 = T([1,2,3])([-0.2,3.9,0.1])(R([2,3])(-PI/2)(STRUCT([
        dettaglio4,
        dettaglio5,
        dettaglio6
    ])))
    ricciolo = STRUCT([ricciolo1,T(1)(0.1)(ricciolo1)])
    # Uniamo il tutto:
    return STRUCT([
        T(3)(-0.2)(finestra),
        TEXTURE('Texture/pietraB.jpg')(corniceEst),
        TEXTURE('Texture/pietra1.jpg')(corniceInt),
        T([1,2])([-0.2,4])(triangolo),
        TEXTURE('Texture/pietra3.jpg')(ricciolo),
        TEXTURE('Texture/pietra3.jpg')(T(1)(2.21)(ricciolo))
    ])

VIEW(finestraFacciata())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEF2DE0> >

![ws7_finGrande.png](files/Immagini/ws7_finGrande.png)

![ws7_finPiccola.png](files/Immagini/ws7_finPiccola.png)

![ws7_finFacciata.png](files/Immagini/ws7_finFacciata.png)

### - Fori per le finestre:

In [52]:
# Aggiungiamo i fori per le finestre:
#
# Finestra grande asse X:
finGX = CUBOID([1,2,0.5])
# Finestra grande asse Z:
finGZ = CUBOID([0.5,2,1])
# Finestra piccola asse X:
finPX = CUBOID([1,1,0.5])
# Finestra piccola asse Z:
finPZ = CUBOID([0.5,1,1])
# Finestra facciata asse X:
finFX = CUBOID([2,4,0.5])
# Tutte le finestre:
buchiFinestrePT = STRUCT([
    #
    # Finestre grandi:
    # - dietro:
    T([1,2])([15.5,2])(finGX),
    T([1,2])([29.5,2])(finGX),
    # - dietro laterale:
    T([1,2,3])([18,2,-2.5])(finGZ),
    T([1,2,3])([27.5,2,-2.5])(finGZ),
    #
    # Finestre piccole:
    # - laterale sx:
    T([2,3])([3,5])(finPZ),
    T([2,3])([3,13])(finPZ),
    T([2,3])([3,18])(finPZ),
    # - laterale dx:
    T([1,2,3])([45.5,3,5])(finPZ),
    T([1,2,3])([45.5,3,13])(finPZ),
    T([1,2,3])([45.5,3,18])(finPZ),
    # - dietro:
    T([1,2])([2,3])(finPX),
    T([1,2])([6.5,3])(finPX),
    T([1,2])([10.5,3])(finPX),
    T([1,2])([34.5,3])(finPX),
    T([1,2])([38.5,3])(finPX),
    T([1,2])([43,3])(finPX),
    #
    # Finestre facciata:
    T([1,2,3])([2.5,2.5,22])(finFX),
    T([1,2,3])([8.5,2.5,22])(finFX),
    T([1,2,3])([35.5,2.5,22])(finFX),
    T([1,2,3])([41.5,2.5,22])(finFX),
])

pianoTerraBuchiFinestre = DIFFERENCE([pianoTerra(h),buchiFinestrePT])

VIEW(pianoTerraBuchiFinestre)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E0EAE70> >

![ws7_foriFinPT.png](files/Immagini/ws7_foriFinPT.png)

### - Inserimento finestre:

In [54]:
# Posizioniamo le finestre nei fori appena fatti:
#
# Creiamo le finestre che si possano mettere sul retro dell'edificio:
finestraGrandeR = R([1,3])(PI)(finestraGrande())
finestraPiccolaR = R([1,3])(PI)(finestraPiccola())
# Creiamo le finestre che si possono mettere sul lato dell'edificio:
finestraGrandeLSX = R([1,3])(PI/2)(finestraGrande())
finestraGrandeLDX = R([1,3])(-PI/2)(finestraGrande())
finestraPiccolaLSX = R([1,3])(PI/2)(finestraPiccola())
finestraPiccolaLDX = R([1,3])(-PI/2)(finestraPiccola())
finestrePT = STRUCT([
    #
    # Finestre retro:
    T([1,2])([16.5,2])(finestraGrandeR),
    T([1,2])([30.5,2])(finestraGrandeR),
    T([1,2])([3,3])(finestraPiccolaR),
    T([1,2])([7.5,3])(finestraPiccolaR),
    T([1,2])([11.5,3])(finestraPiccolaR),
    T([1,2])([35.5,3])(finestraPiccolaR),
    T([1,2])([39.5,3])(finestraPiccolaR),
    T([1,2])([44,3])(finestraPiccolaR),
    #
    # Finestre davanti:
    T([1,2,3])([2.5,2.5,22.5])(finestraFacciata()),
    T([1,2,3])([8.5,2.5,22.5])(finestraFacciata()),
    T([1,2,3])([35.5,2.5,22.5])(finestraFacciata()),
    T([1,2,3])([41.5,2.5,22.5])(finestraFacciata()),
    #
    # Finestre laterali:
    # - sx:
    T([1,2,3])([18,2,-2.5])(finestraGrandeLSX),
    T([2,3])([3,5])(finestraPiccolaLSX),
    T([2,3])([3,13])(finestraPiccolaLSX),
    T([2,3])([3,18])(finestraPiccolaLSX),
    # - dx:
    T([1,2,3])([28,2,-1.5])(finestraGrandeLDX),
    T([1,2,3])([46,3,6])(finestraPiccolaLDX),
    T([1,2,3])([46,3,14])(finestraPiccolaLDX),
    T([1,2,3])([46,3,19])(finestraPiccolaLDX)
])

pianoTerraFinestre = STRUCT([pianoTerraBuchiFinestre,finestrePT])
VIEW(pianoTerraFinestre)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEE2DB0> >

![ws7_insFinPT1.png](files/Immagini/ws7_insFinPT1.png)

![ws7_insFinPT2.png](files/Immagini/ws7_insFinPT2.png)

<hr>

### - Porta ingresso:

In [11]:
# La porta d'ingresso è una porta normale con l'aggiunta
# dei dettagli delle finestre grandi:
def portaIngresso():
    portaI = porta()
    # Creiamo la cornice esterna:
    cornice = STRUCT([
        T([1,3])([-0.05,0.2])(CUBOID([0.05,3,0.01])),
        T([1,3])([1.2,0.2])(CUBOID([0.05,3,0.01])),
        T(1)(-0.2)(CUBOID([0.2,3,0.01])),
        T(1)(-0.01)(CUBOID([0.01,3,0.2])),
        T([1,2])([1.2,0.01])(CUBOID([0.01,3,0.2])),
        T(1)(1.2)(CUBOID([0.2,3,0.01]))
    ])
    # Con MKPOL creiamo la struttura triangolare superiore:
    verts1 = [[0,0,0],[0.2,0,0],[0.8,0.2,0],[1.4,0,0],[1.6,0,0],[0.8,0.3,0]]
    cells = [[1,2,3,6],[3,4,5,6]]
    pols = [[1],[2]]
    dettaglio1= OFFSET([0,0,0.2])(MKPOL([verts1,cells,pols]))
    verts2 = [[0.2,0,0],[0.25,0,0],[0.8,0.2,0],[1.4,0,0],[1.45,0,0],[0.8,0.25,0]]
    dettaglio2 = OFFSET([0,0,0.24])(MKPOL([verts2,cells,pols]))
    dettaglio3 = STRUCT([
        TEXTURE('Texture/pietra3.jpg')(T(1)(0.2)(CUBOID([1.2,0.04,0.2]))),
        TEXTURE('Texture/pietra2.jpg')(T(1)(0.25)(CUBOID([1.15,0.005,0.25]))),
        TEXTURE('Texture/pietra2.jpg')(T([1,2])([0.35,0.04])(CUBOID([0.95,0.005,0.25])))
    ])
    triangolo = STRUCT([
        TEXTURE('Texture/pietra2.jpg')(dettaglio1),
        TEXTURE('Texture/pietra1.jpg')(dettaglio2),
        dettaglio3
    ])
    # Creiamo i riccioli laterali:
    dettaglio4 = DIFFERENCE([T([2,3])([0.06,0.2])(R([1,3])(-PI/2)(CYLINDER([0.06,0.04])(20))),T(2)(0.1)(CUBOID([0.1,0.1,0.27]))])
    dettaglio5 = T([2,3])([0.07,0.03])(R([1,3])(-PI/2)(CYLINDER([0.03,0.04])(20)))
    dettaglio6 = OFFSET([0.04,0,0])(MKPOL([[[0,0.1,0.02],[0,0.005,0.2],[0,0.04,0.2]],[[1,2,3]],[[1]]]))
    ricciolo1 = T([1,2,3])([-0.14,1.75,0.1])(R([2,3])(-PI/2)(STRUCT([dettaglio4,dettaglio5,dettaglio6])))
    ricciolo = STRUCT([ricciolo1,T(1)(0.05)(ricciolo1)])
    # Uniamo il tutto:
    return STRUCT([
        T(3)(-0.3)(portaI),
        TEXTURE('Texture/pietra2.jpg')(cornice),
        T([1,2])([-0.2,3])(triangolo),
        TEXTURE('Texture/pietra3.jpg')(T([1,2])([-0.05,0.95])(ricciolo)),
        TEXTURE('Texture/pietra3.jpg')(T([1,2])([1.44,0.95])(ricciolo))
    ])

VIEW(portaIngresso())

Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E13FB70> >

![ws7_portaIngr.png](files/Immagini/ws7_portaIngr.png)

<hr>

### - Porta retro:

In [12]:
# La porta sul retro è più grande delle porte normali, 
# è fatta di vetro ed è protetta da grate:
def portaRetro():
    portaR1 = CUBOID([2,4,0.5])
    portaR = hex_material("#ffffff", "#ffffff",0.4)(portaR1)
    tuboGrataOrizzontaleGrosso = R([1,3])(-PI/2)(CYLINDER([0.05,2])(20))
    tuboGrataOrizzontaleFino = R([1,3])(-PI/2)(CYLINDER([0.02,2])(20))
    tuboGrataVerticaleGrosso = R([2,3])(-PI/2)(CYLINDER([0.05,4.08])(20))
    serratura = STRUCT([
        TEXTURE('Texture/ferro1.jpg')(T([1,2])([0.85,2])(CUBOID([0.4,0.2,0.05]))),
        TEXTURE('Texture/ottone1.jpg')(T([1,2,3])([1.15,2.1,0.1])(JOIN(SPHERE(0.05)([18,36]))))   
    ])
    grata = TEXTURE('Texture/ferro1.jpg')(STRUCT([
        T(2)(3.4)(tuboGrataOrizzontaleGrosso),
        STRUCT(NN(5)([T(2)(3)(tuboGrataOrizzontaleFino),T(2)(0.2)])),
        T(2)(2)(tuboGrataOrizzontaleFino),
        T(2)(2.2)(tuboGrataOrizzontaleFino),
        T(2)(0.4)(tuboGrataOrizzontaleFino),
        T(2)(0.2)(tuboGrataOrizzontaleFino),
        tuboGrataOrizzontaleGrosso,
        T(2)(4)(tuboGrataOrizzontaleGrosso),
        T(2)(-0.04)(tuboGrataVerticaleGrosso),
        T([1,2])([2,-0.04])(tuboGrataVerticaleGrosso),
        T([1,2])([1,-0.04])(tuboGrataVerticaleGrosso),
    ]))
    return STRUCT([
        T(3)(-0.5)(portaR),
        TEXTURE('Texture/mattoni2.jpg')(T([1,2])([1,2.4])(Arch(2.4,0.4,0.02,0.7*PI))),
        grata,
        serratura,
    ])

VIEW(portaRetro())

Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEE5CC0> >

![ws7_portaRetro.png](files/Immagini/ws7_portaRetro.png)

<hr>

### - Scale interne:

In [13]:
# Scale interne:
def scaleInterne():
    scale1 = T(2)(-0.5)(R([1,3])(PI)(STRUCT(NN(4)([CUBOID([1,0.5,2]),T([1,2])([-0.5,-0.5])]))))
    scale2 = T([1,2,3])([1.5,-2.5,-4])(STRUCT(NN(4)([CUBOID([1,0.5,2]),T([1,2])([-0.5,-0.5])])))
    pianerottolo1 = T([1,3])([-2,-4])(CUBOID([2,0.1,4]))
    pianerottolo2 = T([1,2])([3.5,-2])(pianerottolo1)
    return STRUCT([
        TEXTURE('Texture/pietra2.jpg')(scale1),
        TEXTURE('Texture/pietra2.jpg')(scale2),
        TEXTURE('Texture/pavimento1.jpg')(pianerottolo1),
        TEXTURE('Texture/pavimento1.jpg')(pianerottolo2)
    ])

scaleInterneTot = STRUCT([
    T([1,3])([29.5,13.5])(scaleInterne()),
    T([1,3])([16,9.5])(R([1,3])(PI)((scaleInterne())))
])

VIEW(scaleInterneTot)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E112990> >

![ws7_scaleInt.png](files/Immagini/ws7_scaleInt.png)

<hr>

## - Piano terra completo:

In [14]:
# Uniamo (per facilitare la computazione) tutti i fori da sottrarre alla struttura iniziale:
buchiTotPT = STRUCT([buchiPortePT,buchiFinestrePT])
differenzaPT = DIFFERENCE([pianoTerra(h),buchiTotPT])

VIEW(differenzaPT)

# Pavimento:
cross = STRUCT([
    CUBOID([18,0.1,9.5]),
    T([1,3])([4.5,-4])(CUBOID([9.5,0.1,18]))
])
pavimento = STRUCT([
    TEXTURE('Texture/pavimento2.jpg')(CUBOID([5,0.1,10])),
    TEXTURE('Texture/pavimento1.jpg')(T(1)(4.5)(CUBOID([9.5,0.1,10]))),
    TEXTURE('Texture/pavimento3.jpg')(T(1)(14)(cross)),
    TEXTURE('Texture/pavimento1.jpg')(T(1)(31.5)(CUBOID([10,0.1,10]))),
    TEXTURE('Texture/pavimento2.jpg')(T(1)(41)(CUBOID([4.5,0.1,10]))),
    TEXTURE('Texture/pavimento4.jpg')(T(3)(10)(CUBOID([12.5,0.1,12]))),
    #TEXTURE('Texture/pavimento2.jpg')(T([1,3])([13,10])(CUBOID([5,0.1,4]))),
    #TEXTURE('Texture/pavimento2.jpg')(T([1,3])([28,10])(CUBOID([5,0.1,4]))),
    TEXTURE('Texture/pavimento4.jpg')(T([1,3])([33,10])(CUBOID([12.5,0.1,12])))
])

# Il piano terra è l'unione di tutti gli elementi d'arredo:
pianoTerraTot = STRUCT([
    COLOR([0.9,0.8,0.7])(differenzaPT),
    portePT,
    finestrePT,
    T([1,3])([22.4,14])(portaIngresso()),
    R([1,3])(PI)(T([1,3])([-24,4])(portaRetro())),
    scaleInterneTot,
    pavimento
])

VIEW(pianoTerraTot)

Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E13FD80> >

![ws7_PTCompleto1.png](files/Immagini/ws7_PTCompleto1.png)

![ws7_PTCompleto2.png](files/Immagini/ws7_PTCompleto2.png)

![ws7_PTCompleto3.png](files/Immagini/ws7_PTCompleto3.png)

![ws7_PTCompleto4.png](files/Immagini/ws7_PTCompleto4.png)

<hr>

## Struttura primo piano:

In [15]:
# Utilizziamo la funzione usata per il piano terra per realizzare
# un primo piano con muri più bassi:
pianoPrimo = T(2)(h)(pianoTerra(h-2))

VIEW(STRUCT([pianoTerraTot,pianoPrimo]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E13FC30> >

![ws7_strutturaPP.png](files/Immagini/ws7_strutturaPP.png)

### - Finestre:

In [16]:
# Aggiungiamo un nuovo tipo di finestra:
# La finestra sul retro è simile alla porta sul retro, ma più semplice:
def finestraGrateVetro():
    finestraGV1 = CUBOID([1.5,1.5,0.5])
    finestraGV = hex_material("#ffffff", "#ffffff",0.4)(finestraGV1)
    tuboGrataOrizzontale = R([1,3])(-PI/2)(CYLINDER([0.02,1.53])(20))
    tuboGrataVerticale = R([2,3])(-PI/2)(CYLINDER([0.02,1.5])(20))
    grata = TEXTURE('Texture/ferro1.jpg')(STRUCT([
        STRUCT(NN(6)([T(1)(-0.015)(tuboGrataOrizzontale),T(2)(0.3)])),
        STRUCT(NN(6)([tuboGrataVerticale,T(1)(0.3)])),
    ]))
    return STRUCT([
        TEXTURE('Texture/mattoni2.jpg')(T([1,2,3])([0.75,0.1,-0.02])(Arch(2,0.3,0.02,0.7*PI))),
        finestraGV,
        grata
    ])
    
VIEW(finestraGrateVetro())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E0EDE70> >

![ws7_finGrateV.png](files/Immagini/ws7_finGrateV.png)

### - Fori per le finestre:

In [17]:
# Facciamo i fori del primo piano:
# questi riguardano solo finestre piccole ed una finestra particolare,
# posta sopra alla porta secondaria:
buchiFinestrePP = STRUCT([
    # Finestre piccole:
    # - laterale sx:
    T([2,3])([2,5])(finPZ),
    T([2,3])([2,13])(finPZ),
    T([2,3])([2,18])(finPZ),
    # - laterale dx:
    T([1,2,3])([45.5,2,5])(finPZ),
    T([1,2,3])([45.5,2,13])(finPZ),
    T([1,2,3])([45.5,2,18])(finPZ),
    # - dietro:
    T([1,2])([2,2.8])(finPX),
    T(1)(2)(finPX),
    T([1,2])([6.5,2.8])(finPX),
    T(1)(6.5)(finPX),
    T([1,2])([10.5,2.8])(finPX),
    T(1)(10.5)(finPX),
    T([1,2])([34.5,2.8])(finPX),
    T(1)(34.5)(finPX),
    T([1,2])([38.5,2.8])(finPX),
    T(1)(38.5)(finPX),
    T([1,2])([43,2.8])(finPX),
    T(1)(43)(finPX),
    # Finestra retro:
    T([1,2,3])([22.25,1.5,-4])(CUBOID([1.5,1.5,0.5]))    
])

pianoPrimoBuchiFinestre = DIFFERENCE([pianoPrimo,T(2)(h)(buchiFinestrePP)])

VIEW(pianoPrimoBuchiFinestre)

VIEW(STRUCT([pianoTerraTot,pianoPrimoBuchiFinestre]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E13FF00> >

![ws7_foriFinPP1.png](files/Immagini/ws7_foriFinPP1.png)

![ws7_foriFinPP2.png](files/Immagini/ws7_foriFinPP2.png)

### - Inserimento finestre:

In [18]:
# Posizioniamo le finestre nei fori:
finestrePP = STRUCT([
    #
    # Finestre dietro:
    T([1,2])([3,2.8])(finestraPiccolaR),
    T(1)(3)(finestraPiccolaR),
    T([1,2])([7.5,2.8])(finestraPiccolaR),
    T(1)(7.5)(finestraPiccolaR),
    T([1,2])([11.5,2.8])(finestraPiccolaR),
    T(1)(11.5)(finestraPiccolaR),
    T([1,2])([35.5,2.8])(finestraPiccolaR),
    T(1)(35.5)(finestraPiccolaR),
    T([1,2])([39.5,2.8])(finestraPiccolaR),
    T(1)(39.5)(finestraPiccolaR),
    T([1,2])([44,2.8])(finestraPiccolaR),
    T(1)(44)(finestraPiccolaR),
    #
    # Finestre laterali:
    # - sx:
    T([2,3])([2,5])(finestraPiccolaLSX),
    T([2,3])([2,13])(finestraPiccolaLSX),
    T([2,3])([2,18])(finestraPiccolaLSX),
    # - dx:
    T([1,2,3])([46,2,6])(finestraPiccolaLDX),
    T([1,2,3])([46,2,14])(finestraPiccolaLDX),
    T([1,2,3])([46,2,19])(finestraPiccolaLDX),
    # Finestra retro:
    T([1,2,3])([22.25,1.5,-4.02])(finestraGrateVetro())   
])

pianoPrimoFinestre = STRUCT([
    pianoPrimoBuchiFinestre,
    T(2)(h)(finestrePP)
])

VIEW(pianoPrimoFinestre)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEF2B10> >

![ws7_insFinPP.png](files/Immagini/ws7_insFinPP.png)

<hr>

## - Primo piano completo:

In [19]:
# Unione di piano terra e primo piano:
pianoPrimoTot = COLOR([0.9,0.8,0.7])(pianoPrimoFinestre)

VIEW(STRUCT([pianoTerraTot,pianoPrimoTot]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E13FA50> >

![ws7_PPCompleto1.png](files/Immagini/ws7_PPCompleto1.png)

![ws7_PPCompleto2.png](files/Immagini/ws7_PPCompleto2.png)

<hr>

## Struttura piano sotterraneo:

### - Archi interni:

In [20]:
# Definiammo delle funzioni per la realizzazione di archi:

dominioA = PROD([PROD([INTERVALS(1)(30),INTERVALS(1)(1)]),QUOTE([1])])

def bottomArc(d):
    return BEZIER(S1)([[0,0,0],[0,2*d/3,0],[d,2*d/3,0],[d,0,0]])

def bottomArcProf(d):
    return BEZIER(S1)([[0,0,0.5],[0,2*d/3,0.5],[d,2*d/3,0.5],[d,0,0.5]])

def downArc(d):
    return BEZIER(S1)([[0,0,0],[d,0,0]])

def downArcProf(d):
    return BEZIER(S1)([[0,0,0.5],[d,0,0.5]])

#VIEW(MAP(bottomArc(2))(dominioA))
#VIEW(MAP(bottomArcProf(2))(dominioA))

#VIEW(MAP(downArc(2))(dominioA))
#VIEW(MAP(downArcProf(2))(dominioA))

def arc2DSott(d):
    return BEZIER(S2)([bottomArc(d),downArc(d)])

def arc2DSottProf(d):
    return BEZIER(S2)([bottomArcProf(d),downArcProf(d)])

#VIEW(MAP(arc2DSott(2))(dominioA))
#VIEW(MAP(arc2DSottProf(2))(dominioA))

def arc3DSott(d):
    return BEZIER(S3)([arc2DSott(d),arc2DSottProf(d)])

arcoSotterraneo = STRUCT([
    T(2)(3)(MAP(arc3DSott(2))(dominioA)),
    CUBOID([2,3,0.5])
])

VIEW(arcoSotterraneo)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEF32A0> >

![ws7_archiInt.png](files/Immagini/ws7_archiInt.png)

<hr>

### - Finestre:

In [21]:
# Le finestrelle del piano sotterraneo sono piccole, con grate e senza vetri:
def finestraGrate():
    tuboGrataOrizzontale = R([1,3])(-PI/2)(CYLINDER([0.02,0.9])(20))
    tuboGrataVerticale = R([2,3])(-PI/2)(CYLINDER([0.02,0.9])(20))
    grata = TEXTURE('Texture/ferro1.jpg')(STRUCT([
        STRUCT(NN(2)([T(2)(0.3)(tuboGrataOrizzontale),T(2)(0.3)])),
        STRUCT(NN(2)([T(1)(0.3)(tuboGrataVerticale),T(1)(0.3)])),
    ]))
    corniceExt = TEXTURE('Texture/pietra2.jpg')(STRUCT([
        T([1,2])([-0.15,-0.15])(CUBOID([1.2,0.15,0.01])),
        T([1,2])([-0.15,0.9])(CUBOID([1.2,0.15,0.01])),
        T(1)(-0.15)(CUBOID([0.15,0.9,0.01])),
        T(1)(0.9)(CUBOID([0.15,0.9,0.01]))
    ]))
    corniceInt = TEXTURE('Texture/pietra2.jpg')(STRUCT([
        CUBOID([0.01,0.9,0.5]),
        T(1)(0.89)(CUBOID([0.01,0.9,0.5])),
        CUBOID([0.9,0.01,0.5]),
        T(2)(0.89)(CUBOID([0.9,0.01,0.5]))
    ]))
    return STRUCT([
        T(3)(0.25)(grata),
        T(3)(-0.01)(corniceExt),
        corniceInt
    ])
    
VIEW(finestraGrate())

# Dimensione della finestra, utile per i fori:
finestraG = CUBOID([0.9,0.9,0.5])

![ws7_finGrate.png](files/Immagini/ws7_finGrate.png)

### - Fori per porte e finestre:

In [22]:
# Utilizziamo la funzione usata per il piano terra per realizzare
# un piano sotterraneo con muri più bassi, e usiamo le differenze
# per ottenere dei fori per la creazione di archi di passaggio:
diffSotterraneo = STRUCT([
    T([1,3])([18,9.5])(CUBOID([10,h-4,4])),
    T([1,3])([4.5,6])(R([1,3])(-PI/2)(arcoSotterraneo)),
    T([1,3])([14,6])(R([1,3])(-PI/2)(arcoSotterraneo)),
    T([1,3])([31.5,6])(R([1,3])(-PI/2)(arcoSotterraneo)),
    T([1,3])([41,6])(R([1,3])(-PI/2)(arcoSotterraneo)),
    T([1,3])([8.5,9.5])(arcoSotterraneo),
    T([1,3])([36,9.5])(arcoSotterraneo),
    # porta secondaria:
    T([1,3])([22.3,-4])(CUBOID([1.2,3,0.5])),
    # porta secondaria laterale (sx):
    T(3)(12)(R([1,3])(-PI/2)(CUBOID([1.2,3,0.5]))),
    # finestra grate vetro laterale (sx):
    T([2,3])([1.5,18])(R([1,3])(-PI/2)(CUBOID([1.5,1.5,0.5]))),
    #
    # finestrelle con grate:
    # - retro:
    T([1,2])([3,1.5])(finestraG),
    T([1,2])([9,1.5])(finestraG),
    T([1,2])([15,1.5])(finestraG),
    T([1,2])([30.1,1.5])(finestraG),
    T([1,2])([36.1,1.5])(finestraG),
    T([1,2])([42.1,1.5])(finestraG),
    # - davanti:
    T([1,2,3])([3,1.5,22])(finestraG),
    T([1,2,3])([9,1.5,22])(finestraG),
    T([1,2,3])([36.1,1.5,22])(finestraG),
    T([1,2,3])([42.1,1.5,22])(finestraG)
])
sotterraneo = DIFFERENCE([pianoTerra(h-3),diffSotterraneo])
pianoSotterraneoBuchi = COLOR([0.9,0.8,0.7])(T(2)(-(h-3))(sotterraneo))

VIEW(pianoSotterraneoBuchi)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEF3AB0> >

![ws7_foriPS.png](files/Immagini/ws7_foriPS.png)

<hr>

### - Scala frontale:

In [23]:
# Scala frontale:
def scalaFrontale():
    scale = STRUCT(NN(9)([CUBOID([5,0.5,1.5]),T([2,3])([-0.5,0.75])]))
    colonnina = STRUCT([
        TEXTURE('Texture/mattoni3.jpg')(CUBOID([1,2.5,1])),
        TEXTURE('Texture/pietra1.jpg')(T([1,2,3])([-0.05,2.5,-0.05])(CUBOID([1.1,0.1,1.1]))),
        TEXTURE('Texture/mattoni3.jpg')(T([1,2,3])([0.05,2.6,0.05])(CUBOID([0.9,0.3,0.9]))),
        TEXTURE('Texture/pietra1.jpg')(T([1,2,3])([-0.05,2.9,-0.05])(CUBOID([1.1,0.1,1.1])))
    ])
    muretto = T([1,2])([-0.5,-4])(STRUCT([
        TEXTURE('Texture/pietra1.jpg')(OFFSET([0.5,0,0])(MKPOL([
            [[0,0,0],[0,0,8],[0,2,8],[0,7,3],[0,7,0]],
            [[1,2,3,4,5]],
            [0]
        ]))),
        T([1,3])([-0.25,8])(colonnina)
    ]))
    
    return STRUCT([
        TEXTURE('Texture/mattoni3.jpg')(scale),
        muretto,
        T(1)(5.5)(muretto)
    ])

VIEW(scalaFrontale())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEF36C0> >

![ws7_scalaFront.png](files/Immagini/ws7_scalaFront.png)

<hr>

### - Porta secondaria:

In [24]:
# La porta secondaria è una normale porta con l'archetto di mattoni sopra:
def portaSecondaria():
    return STRUCT([
        porta(),
        TEXTURE('Texture/mattoni2.jpg')(T([1,2,3])([0.58,1.5,-0.05])(Arch(2,0.3,0.02,0.7*PI)))
    ])

VIEW(portaSecondaria())

Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E13F240> >

![ws7_portaSec.png](files/Immagini/ws7_portaSec.png)

<hr>

### - Scala sul retro:

In [26]:
# Scriviamo una funzione per l'ornamento alla fine del corrimano,
# utilizzando le curve di Bezier:
def ricciolo():
    curvaSX = BEZIER(S1)([
        [0,0.05,-0.05],
        [0,0,0],
        [0,0,0.1],
        [0,0.2,0.1],
        [0,0.2,0],
        [0,0.3,-0.1],
    ])
    curvaSpessore = BEZIER(S1)([
        [0.03,0.05,-0.05],
        [0.03,0,0],
        [0.03,0,0.1],
        [0.03,0.2,0.1],
        [0.03,0.2,0],
        [0.03,0.3,-0.1],
    ])
    curva2D = BEZIER(S2)([curvaSX,curvaSpessore]) 
    curvaDX = BEZIER(S1)([
        [-0.03,0.05,-0.05],
        [-0.03,0,0],
        [-0.03,0,0.1],
        [-0.03,0.2,0.1],
        [-0.03,0.2,0],
        [-0.03,0.3,-0.1],
    ])
    curvaSpessore2 = BEZIER(S1)([
        [0,0.05,0],
        [0,0,0.02],
        [0,0,0.12],
        [0,0.2,0.12],
        [0,0.2,0.06],
        [0,0.3,-0.05],
    ])
    curva2D1 = BEZIER(S2)([curvaDX,curvaSpessore2]) 
    
    dominioR1 = PROD([INTERVALS(1)(32),INTERVALS(1)(1)])
    dominioR = PROD([dominioR1,QUOTE([1])])
    curva3D1 = BEZIER(S3)([curva2D,curva2D1]) 

    curva3D = MAP(curva3D1)(dominioR)
    return curva3D
    
VIEW(ricciolo())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEDB6F0> >

![ws7_ricciolo.png](files/Immagini/ws7_ricciolo.png)

In [28]:
# Scala sul retro:
def scalaRetro():
    gradini = STRUCT([
        CUBOID([13.8,0.6,3]),
        T([1,2])([0.6,0.6])(CUBOID([12.6,0.6,3])),
        T([1,2])([1.2,1.2])(CUBOID([11.4,0.6,3])),
        T([1,2])([1.8,1.8])(CUBOID([10.2,0.6,3])),
        T([1,2])([2.4,2.4])(CUBOID([9,0.6,3])),
        T([1,2])([3,3])(CUBOID([7.8,0.6,3])),
        T([1,2])([3.6,3.6])(CUBOID([6.6,0.6,3])),
        #T([1,2])([4.2,4.2])(CUBOID([5.4,0.6,3])),
        #T([1,2])([4.8,4.8])(CUBOID([4.2,0.6,3])),
        #T([1,2])([5.4,5.4])(CUBOID([3,0.6,3]))
    ])
    foro = T(1)(6.2)(CUBOID([1.2,3,3]))
    scalaBucata = DIFFERENCE([gradini,foro])
    dettaglioG1 = T([1,2])([3,-0.6])(CUBOID([0.65,0.05,3.05]))
    dettaglioG2 = STRUCT(NN(6)([dettaglioG1,T([1,2])([0.6,-0.6])]))
    dettaglioGradini = T(3)(-0.05)(STRUCT([
        T(1)(-3.65)(CUBOID([6.7,0.05,3.05])),
        dettaglioG2,
        T([1,3])([-0.6,3.05])(R([1,3])(PI)(dettaglioG2))
    ]))
    # Tubo orizzontale:
    tuboRO = R([1,3])(-PI/2)(CYLINDER([0.03,6.6])(20))
    # Tubo inclinato di 45°:
    tuboR45 = R([1,2])(-PI/4)(R([1,3])(-PI/2)(CYLINDER([0.03,5.6])(20)))
    # Tubo verticale:
    tuboRV = R([2,3])(-PI/2)(CYLINDER([0.02,2])(20))
    corrimano = T([1,2])([10.2,4.2])(STRUCT([
        T(2)(2)(tuboR45),
        STRUCT(NN(7)([tuboRV,T([1,2])([0.6,-0.6])])),
        T([1,2])([3.9,-2.11])(R([1,3])(-PI/2)(ricciolo()))
    ]))
    ringhiera = TEXTURE('Texture/ferro1.jpg')(STRUCT([
        T([1,2])([3.6,6.2])(tuboRO),
        T([1,2])([3.6,4.2])(STRUCT(NN(22)([tuboRV,T(1)(0.3)]))),
        corrimano,
        T(1)(13.8)(R([1,3])(-PI)(corrimano))
        
    ]))
    return STRUCT([
        TEXTURE('Texture/mattoni3M.jpg')(scalaBucata),
        T(1)(6.2)(portaSecondaria()),
        TEXTURE('Texture/pietra2.jpg')(T([1,2])([7.2,4.2])(dettaglioGradini)),
        ringhiera
    ])

VIEW(scalaRetro())

Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E13FE40> >

![ws7_scalaRetro1.png](files/Immagini/ws7_scalaRetro1.png)

![ws7_scalaRetro2.png](files/Immagini/ws7_scalaRetro2.png)

<hr>

## - Piano sotterraneo completo:

In [30]:
# Utilizziamo una variabile per salvare la struttura della finestra con grate ruotata:
finestraGrateR = R([1,3])(PI)(finestraGrate())

# Poniamo finestre e porte alla struttura del piano sotterraneo:
pianoSotterraneo = STRUCT([
    pianoSotterraneoBuchi,
    # porta secondaria laterale (sx):
    T([2,3])([-4,12])(R([1,3])(-PI/2)(porta())),
    # finestra grate vetro laterale (sx):
    T([1,2,3])([-0.01,-2.5,18])(R([1,3])(-PI/2)(finestraGrateVetro())),
    #
    # finestrelle con grate:
    # - retro:
    T([1,2])([3,-2.5])(finestraGrate()),
    T([1,2])([9,-2.5])(finestraGrate()),
    T([1,2])([15,-2.5])(finestraGrate()),
    T([1,2])([30.1,-2.5])(finestraGrate()),
    T([1,2])([36.1,-2.5])(finestraGrate()),
    T([1,2])([42.1,-2.5])(finestraGrate()),
    # - davanti:
    T([1,2,3])([3.9,-2.5,22.5])(finestraGrateR),
    T([1,2,3])([9.9,-2.5,22.5])(finestraGrateR),
    T([1,2,3])([37,-2.5,22.5])(finestraGrateR),
    T([1,2,3])([43,-2.5,22.5])(finestraGrateR)
])

VIEW(pianoSotterraneo)

# Unione di piano terra, primo piano e piano sotterraneo:
pianoSotterraneoTot = STRUCT([
    COLOR([0.9,0.8,0.7])(pianoSotterraneo),
    T([1,2,3])([20.5,0,22.4])(scalaFrontale()),
    T([1,2,3])([16.1,-4,-7])(scalaRetro())
])

VIEW(STRUCT([pianoTerraTot,pianoPrimoTot,pianoSotterraneoTot]))

Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEDB7B0> >

![ws7_PSCompleto1.png](files/Immagini/ws7_PSCompleto1.png)

![ws7_PSCompleto2.png](files/Immagini/ws7_PSCompleto2.png)

![ws7_PSCompleto3.png](files/Immagini/ws7_PSCompleto3.png)

<hr>

## Facciata frontale:

### - Colonne:

In [31]:
# Definiamo una funzione per creare l'ondina della parte superiore della colonna:
def ondina(d):
    davanti = BEZIER(S1)([[0,0.1,0],[0,2*d/3,0],[d,2*d/3,0],[d,0.1,0]])
    dietro = BEZIER(S1)([[0,0,0],[0,2*d/3,0.1],[d,2*d/3,0.1],[d,0,0]])
    unione2D = BEZIER(S2)([dietro,davanti])
    ultimo = BEZIER(S1)([[0,0.07,0],[0.07,2*d/3,0.07],[d-0.07,2*d/3,0.07],[d,0.07,0]])
    unione3D = BEZIER(S3)([unione2D,ultimo])
    return MAP(unione3D)(dominioA)

d = 0.25
VIEW(ondina(d))

# Definiamo una funzione per creare la spirale della parte superiore della colonna:
def spirale(r,h):
    s1 = CYLINDER([r,h])(20)
    s2 = CYLINDER([r-r/5,h])(20)
    s3 = CYLINDER([r-r/3,h])(20)
    s4 = CYLINDER([r/2,h])(20)
    s5 = CYLINDER([r/4,h])(20)
    return STRUCT([
        s1,
        T(3)(alt/2)(s2),
        T(3)(alt)(s3),
        T(3)(alt*3/2)(s4),
        T(3)(alt*2)(s5)
    ])
r = 0.1
alt = 0.01
VIEW(spirale(r,alt))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E13F750> >

![ws7_ondina.png](files/Immagini/ws7_ondina.png)

![ws7_spirale.png](files/Immagini/ws7_spirale.png)

In [32]:
# Colonne (capitello composito):
def colonna():
    # Decoriamo la parte inferiore della colonna:
    trapezio = T(1)(-0.1)(OFFSET([0,0,0.25])(MKPOL([
        [[0,0,0],[1,0,0],[1,0.2,0],[0.9,0.5,0],[0.1,0.5,0],[0,0.2,0]],
        [[1,2,3,4,5,6]],
        [[1]]
    ])))
    triangolo = T([2,3])([0.5,0.1])(OFFSET([0.8,0,0])(MKPOL([
        [[0,0,0],[0,0,0.15],[0,0.15,0]],
        [[1,2,3]],
        [[1]]
    ])))
    base = STRUCT([
        T(1)(-0.1)(CUBOID([1,0.2,0.3])),
        T([1,2])([-0.12,0.2])(CUBOID([1.04,0.05,0.35])),
        T([1,2])([0,0.25])(CUBOID([0.8,0.05,0.25])),
        T([1,2])([-0.05,0.3])(CUBOID([0.9,0.02,0.3])),
        T([1,2])([-0.1,0.32])(CUBOID([1.02,0.05,0.32])),
        T([1,2])([-0.05,0.37])(CUBOID([0.9,0.02,0.3])),
        trapezio,
    ])
    # Decoriamo la parte superiore della colonna:
    ondine = T([1,2,3])([0.08,7.5,0.1])(STRUCT([
        ondina(d),
        T(1)(0.4)(ondina(d)),
        T([1,2])([0.2,0.1])(ondina(d)),
        T([1,2])([-0.2,0.1])(ondina(d)),
        T([1,2])([0.6,0.1])(ondina(d))
    ]))
    spiraleCompleta = STRUCT([
        R([1,3])(3*PI/4)(spirale(r,alt)),
        R([1,3])(7*PI/4)(spirale(r,alt))
    ])
    spirali = STRUCT([
        spiraleCompleta,
        T(1)(0.9)(R([1,3])(PI/2)(spiraleCompleta))
    ])
    tri = OFFSET([0,0,0.1])(MKPOL([
        [[0,0,0],[0.15,0,0],[0.15,-0.4,0]],
        [[1,2,3]],
        [[1]]
    ]))
    decorazione = T([1,2,3])([-0.05,7.8,0.15])(STRUCT([
        spirali,
        T([1,3])([0.05,-0.05])(R([1,3])(-PI/2)(CYLINDER([0.04,0.8])(20))),
        T([1,2,3])([-0.1,0.1,-0.15])(tri),
        T([1,2,3])([1,0.1,-0.05])(R([1,3])(PI)(tri)),
        T([1,2,3])([-0.05,0.1,-0.15])(OFFSET([0,0.08,0])(MKPOL([
            [[0,0,0],[0.4,0,0],[1,0,0],[1.1,0,0.2],[0.5,0,0.1],[-0.1,0,0.2]],
            [[1,2,5,6],[2,3,4,5]],
            [[1],[2]]
        ]))),
        T([1,2,3])([-0.05,0.2,-0.15])(OFFSET([0,0.01,0])(MKPOL([
            [[0,0,0],[0.4,0,0],[1,0,0],[1.1,0,0.2],[0.5,0,0.1],[-0.1,0,0.2]],
            [[1,2,5,6],[2,3,4,5]],
            [[1],[2]]
        ]))),
        T([1,2,3])([0.04,-0.4,-0.15])(CUBOID([0.82,0.01,0.11])),
        T([1,2,3])([0.04,-0.38,-0.15])(CUBOID([0.82,0.03,0.11]))
    ]))
    superiore = STRUCT([
        ondine,
        decorazione
    ])
    # Uniamo il tutto:
    col = STRUCT([
        TEXTURE('Texture/mattoniScoloriti4.jpg')(CUBOID([0.8,8,0.1])),
        TEXTURE('Texture/pietra5.jpg')(base),
        TEXTURE('Texture/pietra5.jpg')(triangolo),
        TEXTURE('Texture/pietra5.jpg')(superiore)
    ])
    return col

VIEW(colonna()) 

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E126F00> >

![ws7_colonne1.png](files/Immagini/ws7_colonne1.png)

![ws7_colonne2.png](files/Immagini/ws7_colonne2.png)

![ws7_colonne3.png](files/Immagini/ws7_colonne3.png)

### - Posizionamento colonne:

In [33]:
villaTot = STRUCT([pianoTerraTot,pianoPrimoTot,pianoSotterraneoTot])

facciataColonne = (STRUCT([
    villaTot,
    T([1,2,3])([0.2,2.2,22.5])((colonna())),
    T([1,2,3])([6.1,2.2,22.5])((colonna())),
    T([1,2,3])([12,2.2,22.5])((colonna())),
    T([1,2,3])([33.2,2.2,22.5])((colonna())),
    T([1,2,3])([39.1,2.2,22.5])((colonna())),
    T([1,2,3])([45,2.2,22.5])((colonna())),
]))

VIEW(facciataColonne)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEDB600> >

![ws7_posColonne.png](files/Immagini/ws7_posColonne.png)

<hr>

### - Dettagli frontali:

In [34]:
# Funzione che costruisce la colonnina, ricevendo in input larghezza e profondità:
def colonnina(l,p):
    base = STRUCT([
        T([1,2])([0.05,0.5])(CUBOID([l+0.1,0.1,p+0.025])),
        CUBOID([l+0.2,0.5,p+0.05])
    ])
    colonnina = STRUCT([
        TEXTURE('Texture/mattoni3.jpg')(CUBOID([l,2.5,p])),
        TEXTURE('Texture/pietra4.jpg')(T([1,2])([-0.1,2.5])(CUBOID([l+0.2,0.05,p+0.1]))),
        TEXTURE('Texture/pietra5.jpg')(T([1,2])([-0.05,2.475])(CUBOID([l+0.1,0.025,p+0.05]))),
        TEXTURE('Texture/pietra5.jpg')(T([1,2])([-0.025,2.45])(CUBOID([l+0.05,0.0175,p+0.025]))),
        TEXTURE('Texture/pietra5.jpg')(T(1)(-0.1)(base))
    ])
    return colonnina

# Funzione che costruisce il muretto superiore:
def murettoSuperiore():
    lastra = TEXTURE('Texture/mattoni2.jpg')(CUBOID([13,2.5,0.01]))
    lastraPiccola = TEXTURE('Texture/pietra5.jpg')(CUBOID([13,0.5,0.02]))
    # Larghezza e profondità colonnina:
    lCol = 1.2
    pCol = 0.4
    # Larghezza e profondità insenatura:
    lIns = 2.5
    pIns = 0.2
    murettoSuperiore1 = STRUCT([
        # 3 colonnine:
        colonnina(lCol,pCol),
        T(1)(5.9)(colonnina(lCol,pCol)),
        T(1)(11.8)(colonnina(lCol,pCol)),
        # 2 insenature:
        T(1)(2.25)(colonnina(lIns,pIns)),
        T(1)(8.25)(colonnina(lIns,pIns)),
        lastra,
        lastraPiccola
    ])
    murettoSuperiore = STRUCT([
        murettoSuperiore1,
        T(1)(33)(murettoSuperiore1)
    ])
    return murettoSuperiore

#VIEW(murettoSuperiore())

facciataMurettoSuperiore = STRUCT([facciataColonne,T([2,3])([-0.3,22.5])(murettoSuperiore())])

VIEW(facciataMurettoSuperiore)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E0F1FC0> >

![ws7_murettoSup.png](files/Immagini/ws7_murettoSup.png)

In [35]:
# Funzione che costruisce il muretto inferiore:
def murettoInferiore():
    bordo = TEXTURE('Texture/pietra2.jpg')(CUBOID([20.1,0.2,0.5]))
    lastra = STRUCT([
        CUBOID([20,1.35,0.01]),
        T(2)(2.5)(CUBOID([20,1.35,0.01])),
        T(2)(1.35)(CUBOID([2.85,1.35,0.01])),
        T([1,2])([4.05,1.35])(CUBOID([4.8,1.35,0.01])),
        T([1,2])([10.05,1.35])(CUBOID([10,1.35,0.01]))
    ])
    murettoInferiore1 = STRUCT([
        T(1)(-0.1)(bordo),
        TEXTURE('Texture/mattoni1.jpg')(T(2)(-3.7)(lastra))
    ])
    murettoInferiore2 = STRUCT([
        T(1)(26)(bordo),
        TEXTURE('Texture/mattoni1.jpg')(R([1,3])(-PI)(T([1,2,3])([-46,-3.7,-0.05])(lastra)))
    ])
    murettoInferiore = STRUCT([
        murettoInferiore1,
        murettoInferiore2
    ])
    return murettoInferiore

facciataMurettoInferiore = STRUCT([
    facciataColonne,
    T([2,3])([-0.3,22.5])(murettoSuperiore()),
    T([2,3])([-0.3,22.5])(murettoInferiore())
])

VIEW(facciataMurettoInferiore)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEF3DB0> >

![ws7_murettoInf.png](files/Immagini/ws7_murettoInf.png)

<hr>

### - Archi ingresso:

In [36]:
# COLONNA COMPLETA:
colonnaCompleta = STRUCT([
    T([1,2])([0.2,-0.05])(colonna()),
    T(2)(-2.55)(colonnina(1.2,0.4))
])

# Creiamo una struttura ad arco:
#def bottomArcFacciataProf(d):
#    return BEZIER(S1)([[0,0,0.1],[0,2*d/3,0.1],[d,2*d/3,0.1],[d,0,0.1]])

#VIEW(MAP(bottomArc(8))(dominioA))
#VIEW(MAP(bottomArcFacciataProf(8))(dominioA))

def topArc(d):
    return BEZIER(S1)([[0,2*d/3,0],[d,2*d/3,0]])

def topArcProf(d):
    return BEZIER(S1)([[0,2*d/3,0.5],[d,2*d/3,0.5]])

#VIEW(MAP(topArc(8))(dominioA))
#VIEW(MAP(topArcProf(8))(dominioA))

def arc2D(d):
    return BEZIER(S2)([bottomArc(d),topArc(d)])

def arc2DProf(d):
    return BEZIER(S2)([bottomArcProf(d),topArcProf(d)])

#VIEW(MAP(arc2D(8))(dominioA))
#VIEW(MAP(arc2DProf(8))(dominioA))

def arc3D(d):
    return BEZIER(S3)([arc2D(d),arc2DProf(d)])

VIEW(MAP(arc3D(8))(dominioA))

# Costruiamo ora la facciata formata da archi e colonne:
def arcoFacciata():
    arcoBase = MAP(arc3D(5))(dominioA)
    trapezio = OFFSET([0,0,0.6])(MKPOL([
        [[0,0.8,0],[0.2,0,0],[0.5,0,0],[0.7,0.8,0]],
        [[1,2,3,4]],
        [[1]]
    ]))
    return TEXTURE('Texture/mattoniScoloriti7.jpg')(STRUCT([
        T([1,2])([-0.57,-2.55])(CUBOID([0.57,10.5,0.5])),
        T(2)(4.95)(arcoBase),
        T([1,2])([5,-2.55])(CUBOID([0.57,10.5,0.5])),
        T([1,2])([2.15,7.15])(trapezio),
        T([1,2])([-0.57,5])(CUBOID([0.7,0.3,0.6])),
        T([1,2])([4.86,5])(CUBOID([0.7,0.3,0.6]))
    ]))

murettoFacciata = STRUCT([
    TEXTURE('Texture/mattoniScoloriti1.jpg')(CUBOID([5,1.5,0.4])),
    TEXTURE('Texture/pietra2.jpg')(T(2)(1.5)(CUBOID([5,0.2,0.5])))
])

facciataArchi = STRUCT([
    T([1,3])([0.57,-0.5])(arcoFacciata()),
    T([1,2,3])([0.57,-2.5,-0.5])(murettoFacciata),
    T(1)(0.57+5+0.57-0.2)(colonnaCompleta),
    T([1,3])([0.57+5+0.57+0.8+0.57,-0.5])(arcoFacciata()),
    T([1,2,3])([14.5,-2.5,-0.5])(murettoFacciata),
    T(1)(0.57+5+0.57+0.8+0.57+5+0.57-0.2)(colonnaCompleta),
    T([1,3])([0.57+5+0.57+0.8+0.57+5+0.57+0.8+0.57,-0.5])(arcoFacciata()),
    T([1,2,3])([0,-2.2,-9])(TEXTURE('Texture/pietra4.jpg')(CUBOID([20,0.01,8.9])))
])

VIEW(facciataArchi)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEED8D0> >

![ws7_archiIngr1.png](files/Immagini/ws7_archiIngr1.png)

![ws7_archiIngr2.png](files/Immagini/ws7_archiIngr2.png)

![ws7_archiIngr3.png](files/Immagini/ws7_archiIngr3.png)

<hr>

### - Facciata frontale completa:

In [37]:
# Facciata frontale completa:
dettagliLineaSopra = STRUCT([
    COLOR([0.9,0.8,0.7])(CUBOID([46,1.8,0.02])),
    TEXTURE('Texture/pietraBL.jpg')(T(1)(-0.03)(CUBOID([46.06,0.1,0.04]))),
    TEXTURE('Texture/mattoniScoloriti9.jpg')(T([1,2])([-0.03,0.1])(CUBOID([46.06,0.2,0.06]))),
    TEXTURE('Texture/pietraBL.jpg')(T([1,2])([-0.03,0.3])(CUBOID([46.06,0.4,0.08]))),
    TEXTURE('Texture/mattoniScoloriti9.jpg')(T([1,2])([-0.03,0.7])(CUBOID([46.06,0.8,0.10]))),
    TEXTURE('Texture/pietra4.jpg')(T([1,2])([-0.03,1.5])(CUBOID([46.06,0.1,0.15])))
])

lastraFacciata = T([2,3])([2.2,22.5])(TEXTURE('Texture/mattoniScoloriti9.jpg')(STRUCT([
    CUBOID([2.3,8,0.01]),
    T([1,2])([2.3,4.5])(CUBOID([2.45,3.5,0.01])),
    T(1)(4.75)(CUBOID([3.5,8,0.01])),
    T([1,2])([8.2,4.5])(CUBOID([2.55,3.5,0.01])),
    T(1)(10.7)(CUBOID([2.3,8,0.01]))
])))

facciataFrontaleCompleta = STRUCT([
    T([1,2,3])([13,2.25,22.5])(facciataArchi),
    facciataMurettoInferiore,
    T([2,3])([10.2,22.5])(dettagliLineaSopra),
    lastraFacciata,
    T([1,3])([46,45.01])(R([1,3])(-PI)(lastraFacciata))
])

VIEW(facciataFrontaleCompleta)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEEDCC0> >

![ws7_facciataFront1.png](files/Immagini/ws7_facciataFront1.png)

![ws7_facciataFront2.png](files/Immagini/ws7_facciataFront2.png)

## Facciate laterali e posteriore:

### - Facciata laterale sx:

In [38]:
# Facciata laterale sinistra:
facciataLSX = TEXTURE('Texture/mattoni3M.jpg')(T(1)(-0.01)(STRUCT([
    T(2)(-4)(CUBOID([0.01,16,4.8])),
    T([2,3])([10.1,4.8])(CUBOID([0.01,1.9,17.7])),
    T([2,3])([4.1,4.8])(CUBOID([0.01,4.8,17.7])),
    T([2,3])([-4,4.8])(CUBOID([0.01,6.9,6])),
    T([2,3])([-4,12])(CUBOID([0.01,6.9,4.5])),
    T([2,3])([-4,16.5])(CUBOID([0.01,1.5,6])),
    T([2,3])([-3,18])(CUBOID([0.01,5.9,4.5])),
    T([2,3])([8.9,6.2])(CUBOID([0.01,1.2,6.6])),
    T([2,3])([2.9,6.2])(CUBOID([0.01,1.2,6.6])),
    T([2,3])([8.9,14.2])(CUBOID([0.01,1.2,3.6])),
    T([2,3])([2.9,14.2])(CUBOID([0.01,1.2,3.6])),
    T([2,3])([8.9,19.2])(CUBOID([0.01,1.2,3.3])),
    T([2,3])([2.9,19.2])(CUBOID([0.01,1.2,3.3])),
    T([2,3])([-1,10.8])(CUBOID([0.01,3.9,1.2])),
    T([2,3])([-1,16.5])(CUBOID([0.01,3.9,2]))
])))

VIEW(facciataLSX)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E1229C0> >

![ws7_facciataLatSx.png](files/Immagini/ws7_facciataLatSx.png)

### - Facciata laterale dx:

In [39]:
# Facciata laterale destra:
facciataLDX = TEXTURE('Texture/mattoni3M.jpg')(T(1)(46)(STRUCT([
    T(2)(-4)(CUBOID([0.01,16,4.8])),
    T([2,3])([10.1,4.8])(CUBOID([0.01,1.9,17.7])),
    T([2,3])([4.1,4.8])(CUBOID([0.01,4.8,17.7])),
    T([2,3])([-4,4.8])(CUBOID([0.01,6.9,17.7])),
    T([2,3])([-4,16.5])(CUBOID([0.01,1.5,6])),
    T([2,3])([8.9,6.2])(CUBOID([0.01,1.2,6.6])),
    T([2,3])([2.9,6.2])(CUBOID([0.01,1.2,6.6])),
    T([2,3])([8.9,14.2])(CUBOID([0.01,1.2,3.6])),
    T([2,3])([2.9,14.2])(CUBOID([0.01,1.2,3.6])),
    T([2,3])([8.9,19.2])(CUBOID([0.01,1.2,3.3])),
    T([2,3])([2.9,19.2])(CUBOID([0.01,1.2,3.3]))
])))

VIEW(facciataLDX)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E122B10> >

![ws7_facciataLatDx.png](files/Immagini/ws7_facciataLatDx.png)

### - Facciata posteriore:

In [ ]:
# Parte laterale della facciata posteriore:
facciataPosterioreL = TEXTURE('Texture/mattoni3M.jpg')(T([2,3])([-4,-0.01])(STRUCT([
    CUBOID([2.85,6.9,0.01]),
    T(2)(6.9)(CUBOID([1.8,9.1,0.01])),
    T([1,2])([2.85,2.55])(CUBOID([15.2,3.45,0.01])),
    T([1,2])([2.85,6])(CUBOID([12.51,0.9,0.01])),
    T([1,2])([16.64,6])(CUBOID([1.5,0.9,0.01])),
    T([1,2])([2.85,0])(CUBOID([15.2,1.35,0.01])),
    T([1,2])([4.05,1.35])(CUBOID([4.8,1.2,0.01])),
    T([1,2])([10.05,1.35])(CUBOID([4.8,1.2,0.01])),
    T([1,2])([16.05,1.35])(CUBOID([2,1.2,0.01])),
    T([1,2])([1.8,8.1])(CUBOID([16.2,2.8,0.01])),
    T([1,2])([1.8,12.1])(CUBOID([16.2,1.6,0.01])),
    T([1,2])([1.8,14.9])(CUBOID([16.2,1.1,0.01])),
    T([1,2])([3.2,6.9])(CUBOID([3.1,1.2,0.01])),
    T([1,2])([3.2,10.9])(CUBOID([3.1,1.2,0.01])),
    T([1,2])([3.2,13.7])(CUBOID([3.1,1.2,0.01])),
    T([1,2])([7.7,6.9])(CUBOID([2.6,1.2,0.01])),
    T([1,2])([7.7,10.9])(CUBOID([2.6,1.2,0.01])),
    T([1,2])([7.7,13.7])(CUBOID([2.6,1.2,0.01])),
    T([1,2])([11.7,6.9])(CUBOID([3.66,1.2,0.01])),
    T([1,2])([16.64,6.9])(CUBOID([1.5,1.2,0.01])),
    T([1,2])([15.35,8])(CUBOID([2,0.1,0.01])),
    T([1,2])([11.7,10.9])(CUBOID([6.3,1.2,0.01])),
    T([1,2])([11.7,13.7])(CUBOID([6.3,1.2,0.01]))
])))

# Parte sporgente della facciata posteriore:
# - laterale:
facciataPosterioreSL = STRUCT([
    CUBOID([0.01,16,1.36]),
    T(3)(2.64)(CUBOID([0.01,16,1.36])),
    T(3)(1.36)(CUBOID([0.01,6,1.29])),
    T([2,3])([8,1.36])(CUBOID([0.01,8,1.29]))
])
# - dietro:
facciataPosterioreSD = STRUCT([
    CUBOID([4,8,0.01]),
    T(1)(6.01)(CUBOID([4,8,0.01])),
    T(2)(8)(CUBOID([10.01,4.5,0.01])),
    T(2)(12.5)(CUBOID([4.25,3.5,0.01])),
    T([1,2])([4.25,14])(CUBOID([2,2,0.01])),
    T([1,2])([5.77,12.5])(CUBOID([4.25,3.5,0.01]))
])

facciataPosterioreSTot = T([1,2,3])([17.99,-4,-4])(STRUCT([
    TEXTURE('Texture/mattoni3R.jpg')(facciataPosterioreSL),
    TEXTURE('Texture/mattoni3R.jpg')(T([1,3])([10.02,4])(R([1,3])(PI)(facciataPosterioreSL))),
    TEXTURE('Texture/mattoni3R.jpg')(T(3)(-0.01)(facciataPosterioreSD))
]))

# Facciata posteriore completa:
facciataPosteriore = STRUCT([
    facciataPosterioreL,
    T([1,3])([46,-0.01])(R([1,3])(PI)(facciataPosterioreL)),
    facciataPosterioreSTot
])

VIEW(facciataPosteriore)

![ws7_facciataPost.png](files/Immagini/ws7_facciataPost.png)

### - Facciata completa:

In [41]:
facciataCompleta = STRUCT([facciataFrontaleCompleta,facciataLSX,facciataLDX,facciataPosteriore])
VIEW(facciataCompleta)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000DEF5570> >

![ws7_facciataCompl1.png](files/Immagini/ws7_facciataCompl1.png)

![ws7_facciataCompl2.png](files/Immagini/ws7_facciataCompl2.png)

<hr>

## Tetto:

### - Tetto orizzontale:

In [42]:
# Con MKPOL facciamo il triangolo al quale aggiungeremo la profondità con OFFSET:
triangoloTettoO = MKPOL([
    [[0,0,-0.5],[0,0,23],[0,5,11.25]],
    [[1,2,3]],
    [[1]]
])

triangoloTettoOLinea = MKPOL([
    [[0,0,-0.5],[0,0,0],[0,4.5,11.25],[0,0,22.5],[0,0,23],[0,5,11.25]],
    [[1,2,3,6],[3,4,5,6]],
    [[1],[2]]
])

tettoOrizzontale = T([1,2])([-0.5,12])(STRUCT([
    TEXTURE('Texture/mattoni3M.jpg')(T(1)(-0.01)(triangoloTettoO)),
    TEXTURE('Texture/pietraN.jpg')(T(1)(-0.02)(triangoloTettoOLinea)),
    TEXTURE('Texture/tegole2L.jpg')(OFFSET([47,0,0])(triangoloTettoO)),
    TEXTURE('Texture/mattoni3M.jpg')(T(1)(47.01)(triangoloTettoO)),
    TEXTURE('Texture/pietraN.jpg')(T(1)(47.02)(triangoloTettoOLinea))
]))

VIEW(tettoOrizzontale)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E0C1DE0> >

![ws7_tettoOr.png](files/Immagini/ws7_tettoOr.png)

### - Tetto frontale:

In [43]:
# Sempre con MKPOL facciamo la parte del tetto posto frontalmente:
triangoloTettoFGrande = MKPOL([
    [[0,0,0],[23,0,0],[11.5,4,0]],
    [[1,2,3]],
    [[1]]
])

triangoloTettoFPiccolo = MKPOL([
    [[0.5,0,0],[22.5,0,0],[11.5,3.5,0]],
    [[1,2,3]],
    [[1]]
])

triangoloTettoFLinea = MKPOL([
    [[0,0,0],[0.5,0,0],[11.5,3.5,0],[22.5,0,0],[23,0,0],[11.5,4,0]],
    [[1,2,3,6],[3,4,5,6]],
    [[1,2]]
])

tettoFrontale = T([1,2,3])([11.5,12.01,13])(STRUCT([
    TEXTURE('Texture/pietra2.jpg')(T(3)(10.01)(triangoloTettoFPiccolo)),
    TEXTURE('Texture/tegole2L.jpg')(OFFSET([0,0,10])(triangoloTettoFGrande)),
    TEXTURE('Texture/pietra4.jpg')(OFFSET([0,0,0.1])(T(3)(10.01)(triangoloTettoFLinea)))
]))

VIEW(STRUCT([tettoFrontale,tettoOrizzontale]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E122B70> >

![ws7_tettoFront.png](files/Immagini/ws7_tettoFront.png)

### - Tetto retro:

In [44]:
triangoloTettoRetroLinea = MKPOL([
    [[0,0,0],[0.5,0,0],[5.5,2.5,0],[10.5,0,0],[11,0,0],[5.5,3,0]],
    [[1,2,3,6],[3,4,5,6]],
    [[1]]
])

triangoloTettoRetro = MKPOL([
    [[0,0,0],[11,0,0],[5.5,3,0]],
    [[1,2,3]],
    [[1]]
])

tettoRetro = T([1,2,3])([17.5,12,-4.5])(STRUCT([
    TEXTURE('Texture/tegole2L.jpg')(OFFSET([0,0,11])(triangoloTettoRetro)),
    TEXTURE('Texture/pietraB.jpg')(T(3)(-0.01)(triangoloTettoRetro)),
    TEXTURE('Texture/pietraN.jpg')(T(3)(-0.05)((OFFSET([0,0,0.05]))(triangoloTettoRetroLinea)))
]))

VIEW(STRUCT([tettoFrontale,tettoOrizzontale,tettoRetro]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E122A20> >

![ws7_tettoRetro.png](files/Immagini/ws7_tettoRetro.png)

### - Tettoia porta laterale:

In [45]:
triangoloTettoiaLaterale = MKPOL([
    [[0,0,0],[0,0,0.25],[0,1.25,2],[0,0,3.75],[0,0,4],[0,1.5,2]],
    [[1,2,3,6],[3,4,5,6]],
    [[1]]
])

trave = TEXTURE('Texture/pietra2.jpg')(CUBOID([2.4,0.1,0.1]))

tettoiaLaterale = T([1,3])([-2.5,9.5])(STRUCT([
    TEXTURE('Texture/tegole3.jpg')(OFFSET([2.5,0,0])(triangoloTettoiaLaterale)),
    T([1,3])([0.1,0.26])(trave),
    T([1,2,3])([0.1,0.55,1.05])(trave),
    T([1,2,3])([0.1,1.15,1.95])(trave),
    T([1,2,3])([0.1,0.55,2.85])(trave),
    T([1,3])([0.1,3.64])(trave),
]))

VIEW(tettoiaLaterale)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E0C1F60> >

![ws7_tettoia1.png](files/Immagini/ws7_tettoia1.png)

![ws7_tettoia2.png](files/Immagini/ws7_tettoia2.png)

### - Tetto completo:

In [46]:
tettoCompleto = STRUCT([
    facciataCompleta,
    tettoFrontale,
    tettoOrizzontale,
    tettoRetro,
    tettoiaLaterale
])

VIEW(tettoCompleto)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E0C1F90> >

![ws7_tettoCompl1.png](files/Immagini/ws7_tettoCompl1.png)

![ws7_tettoCompl2.png](files/Immagini/ws7_tettoCompl2.png)

<hr>

# Camini:

In [47]:
# Funzione per creare un camino (h è la sua altezza):
def camino(h):
    # Costruiamo la griglia della parte superiore del camino:
    baseGriglia = STRUCT([
        CUBOID([1,0.05,0.05]),
        CUBOID([0.05,0.05,0.5]),
        T(3)(0.45)(CUBOID([1,0.05,0.05])),
        T(1)(0.95)(CUBOID([0.05,0.05,0.5]))
    ])
    parteXGriglia = STRUCT([
        STRUCT(NN(6)([CUBOID([0.05,0.4,0.05]),T(1)(0.19)]))
    ])
    parteZGriglia = STRUCT([
        T(3)(0.15)(CUBOID([0.05,0.4,0.05])),
        T(3)(0.3)(CUBOID([0.05,0.4,0.05]))
    ])
    griglia = TEXTURE('Texture/pietra1.jpg')(STRUCT([
        baseGriglia,
        T(2)(0.4)(baseGriglia),
        parteXGriglia,
        T(3)(0.45)(parteXGriglia),
        parteZGriglia,
        T(1)(0.95)(parteZGriglia)
    ]))
    # Realizziamo la parte triangolare superiore:
    triangolo = MKPOL([
        [[0,0,0],[1,0,0],[0.5,0.4,0]],
        [[1,2,3]],
        [[1]]
    ])
    # Realizziamo la piccola tettoia:
    tettoia = TEXTURE('Texture/tegole3.jpg')(STRUCT([
        T([2,3])([h+0.45,-0.05])(OFFSET([0,0,0.6])(MKPOL([
            [[-0.05,0,0],[0,0,0],[0.5,0.4,0],[1,0,0],[1.05,0,0],[0.5,0.45,0]],
            [[1,2,3,6],[3,4,5,6]],
            [[1]]
        ]))),
        T([1,2,3])([0.46,h+0.85,-0.05])(CUBOID([0.08,0.08,0.6]))
    ]))
    # Uniamo il tutto:
    camino = STRUCT([
        TEXTURE('Texture/pietraB.jpg')(CUBOID([1,h,0.5])),
        T(2)(h)(griglia),
        T(2)(h+0.45)(TEXTURE('Texture/pietraB.jpg')(OFFSET([0,0,0.5])(triangolo))),
        tettoia
    ])
    return camino

VIEW(camino(2))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E0EA0F0> >

![ws7_camino.png](files/Immagini/ws7_camino.png)

### - Tetto con camini:

In [50]:
tettoCompletoCamini = STRUCT([
    tettoCompleto,
    T([1,2,3])([5,13,20])(camino(2)),
    T([1,2,3])([40,13,20])(camino(2)),
    T([1,2,3])([35,15.5,8])(R([1,3])(PI/2)(camino(1.5))),
    T([1,2,3])([10,15.5,8])(R([1,3])(PI/2)(camino(1.5))),
    T([1,2,3])([5,13,2])(camino(1))
])

VIEW(tettoCompletoCamini)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E13FDE0> >

![ws7_tettoCamini.png](files/Immagini/ws7_tettoCamini.png)

<hr>

# Villa completa:

In [51]:
# Aggiungi pavimento (unico) piano sotterraneo + erba intorno alla villa
pavimentoSotterraneo = T(2)(-4.01)(TEXTURE('Texture/pavimentoS2L.jpg')(STRUCT([
    CUBOID([46,0.01,22.5]),
    T([1,3])([18,-7])(CUBOID([10,0.01,7]))
])))
villaCompleta = STRUCT([
    tettoCompletoCamini,
    pavimentoSotterraneo,
    TEXTURE('Texture/prato2.jpg')(T([1,2,3])([-18,-4.02,-20])(CUBOID([80,0.01,60])))
])

VIEW(villaCompleta)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E0F1ED0> >

![ws7_villaCompl1.png](files/Immagini/ws7_villaCompl1.png)

![ws7_villaCompl2.png](files/Immagini/ws7_villaCompl2.png)

![ws7_villaCompl3.png](files/Immagini/ws7_villaCompl3.png)

![ws7_villaCompl4.png](files/Immagini/ws7_villaCompl4.png)

![ws7_villaCompl5.png](files/Immagini/ws7_villaCompl5.png)

![ws7_villaCompl6.png](files/Immagini/ws7_villaCompl6.png)

![ws7_villaCompl7.png](files/Immagini/ws7_villaCompl7.png)

![ws7_villaCompl8.png](files/Immagini/ws7_villaCompl8.png)

![ws7_villaCompl9.png](files/Immagini/ws7_villaCompl9.png)

![ws7_villaCompl10.png](files/Immagini/ws7_villaCompl10.png)